In [41]:
import pandas as pd
import geopandas as gpd

In [42]:
data = pd.read_csv("../data/interim/housing_data.csv")
key = pd.read_csv("../data/processed/Neighborhood_alt_key.csv")
key.set_index("alt_names_list",inplace=True)

In [43]:
columns =["Neighborhood_Name","Renter_housing_units_with_gross_rent","Gross_Rent_Less_Than_$500","Gross_Rent_$500-$999","Gross_Rent_$1,000-$1,499","Gross_Rent_$1,500-$1,999","Gross_Rent_$2,000-$2,499","Gross_Rent_$2,500-$2,999","Gross_Rent_$3,000-$3,499","Gross_Rent_$3,500_or_more","Owner_housing_units_with_a_mortgage","Owner_Costs_Less_Than_$1,000","Owner_Costs_$1,000-$1,499","Owner_Costs_$1,500-$1,999","Owner_Costs_$2,000-$2,499","Owner_Costs_$2,500-$2,999","Owner_Costs_$3,000-$3,499","Owner_Costs_$3,500-$3,999",	"Owner_Costs_$4,000_or_more"]
costs = data[columns]
costs['Neighborhood_Name'] = costs['Neighborhood_Name'].apply(lambda x: str(x).replace(" ",""))
costs.head()

C:\Users\colin\AppData\Local\Temp\ipykernel_368260\1431616706.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  costs['Neighborhood_Name'] = costs['Neighborhood_Name'].apply(lambda x: str(x).replace(" ",""))


,Neighborhood_Name,Renter_housing_units_with_gross_rent,Gross_Rent_Less_Than_$500,Gross_Rent_$500-$999,"Gross_Rent_$1,000-$1,499","Gross_Rent_$1,500-$1,999","Gross_Rent_$2,000-$2,499","Gross_Rent_$2,500-$2,999","Gross_Rent_$3,000-$3,499","Gross_Rent_$3,500_or_more",Owner_housing_units_with_a_mortgage,"Owner_Costs_Less_Than_$1,000","Owner_Costs_$1,000-$1,499","Owner_Costs_$1,500-$1,999","Owner_Costs_$2,000-$2,499","Owner_Costs_$2,500-$2,999","Owner_Costs_$3,000-$3,499","Owner_Costs_$3,500-$3,999","Owner_Costs_$4,000_or_more"
0,CouncilDistrict1,19793,961,687,4092,5159,3772,2568,1183,1371,21177,225,700,2397,2529,3747,3651,2926,5002
1,CouncilDistrict2,21623,2471,2077,4698,4503,3565,2056,1148,1105,14572,212,370,1700,2311,2311,2511,1916,3241
2,CouncilDistrict3,38565,1888,1381,5879,10768,8432,5304,2122,2791,14356,335,293,867,1469,1765,1845,1620,6162
3,CouncilDistrict4,23153,324,1067,4767,6043,5524,2007,1276,2145,12777,67,252,721,1165,1544,1936,1205,5887
4,CouncilDistrict5,23448,1268,955,4131,7763,4862,1970,1230,1269,16927,130,501,1419,2661,3083,2561,1726,4846


In [44]:
costs_merge = costs.join(key, on='Neighborhood_Name', how='inner', rsuffix='_alt_key')
columns.append('L_HOOD')
costs_merge = costs_merge[columns].groupby("L_HOOD").agg(sum)
costs_merge.to_csv("../data/processed/costs.csv")

In [45]:
map_data = gpd.read_file('../data/raw/Neighborhood_Map_Atlas_Districts.geojson')

test = map_data.set_index('L_HOOD').join(costs_merge, rsuffix='_costs')

In [46]:
test.to_file('../data/processed/costs_in_map.geojson', index=True)